# Conversor de Moedas — Notebook de desenvolvimento e testes
Este notebook organiza um conjunto de células para preparar o ambiente, inspecionar os arquivos fonte (`servidor.py` e `cliente.py`), executar testes unitários e de integração, simular concorrência com múltiplos clientes e limpar os recursos ao final.

Siga as seções na ordem para garantir que o servidor esteja em um estado limpo antes de executar testes de integração/concorrência.

## 1) Preparar ambiente e dependências
Instale (se necessário) as dependências usadas neste notebook.
Este projeto usa `requests` para chamadas HTTP e `pytest` para testes. Opcionalmente usamos `requests-mock` e `pandas` para análise.

In [ ]:
# Execute esta célula somente se precisar instalar dependências
# No PowerShell (Windows) você pode executar `pip install ...` diretamente ou usar esta célula do notebook
!pip install requests pytest requests-mock pandas -q
# Imports padrão usados pelas células seguintes
import os, sys, time, socket, subprocess, threading, importlib, json
from pathlib import Path
base_path = Path(r"c:\Users\anaca\OneDrive\Documentos\projetos\camada-aplicacao-socket")
server_path = base_path / 'servidor.py'
client_path = base_path / 'cliente.py'

## 2) Importar e inspecionar os arquivos fonte (servidor.py, cliente.py)
Lemos o conteúdo dos arquivos para revisão rápida sem executar o código do servidor.

In [ ]:
for p in (server_path, client_path):
    print('
' + '='*80)
    print('Arquivo:', p)
    try:
        with open(p, 'r', encoding='utf-8') as f:
            content = f.read()
            print('
'.join(content.splitlines()[:200]))
    except Exception as e:
        print('Erro lendo', p, e)

## 3) Carregar módulo servidor dinamicamente (importlib) e testar funções internas
Para testar funções sem iniciar o servidor, carregamos o módulo com importlib.util.spec_from_file_location.

In [ ]:
import importlib.util
def load_module_from_path(path, name='servidor_mod'):
    spec = importlib.util.spec_from_file_location(name, str(path))
    mod = importlib.util.module_from_spec(spec)
    loader = spec.loader
    if loader is None:
        raise ImportError('Não foi possível carregar o módulo')
    loader.exec_module(mod)
    return mod

# Carregar o módulo servidor sem iniciar o loop de aceitação (o arquivo atual executa código ao importar),
# portanto carregamos com cuidado em um namespace separado.
try:
    servidor_mod = load_module_from_path(server_path, 'servidor_dynamic')
    print('Servidor carregado:', hasattr(servidor_mod, 'convert_currency'))
    # Testar chamadas diretas (cuidado: get_exchange_rates pode fazer chamadas HTTP)
    if hasattr(servidor_mod, 'get_fallback_rates'):
        print('Fallback sample:', servidor_mod.get_fallback_rates())
except Exception as e:
    print('Import dinâmico falhou (esperado se o módulo executa servidor):', e)

## 4) Iniciar servidor em background (subprocess) e capturar saída
Iniciamos `servidor.py` em um subprocess para testes de integração. A saída do servidor é lida em uma thread para que possamos ver logs.
Use `server_proc` para encerrar o servidor com `server_proc.terminate()` ou use a célula de encerramento ao final.

In [ ]:
import subprocess, threading
server_proc = None
def start_server():
    global server_proc
    if server_proc and server_proc.poll() is None:
        print('Servidor já em execução (PID):', server_proc.pid)
        return server_proc
    cmd = [sys.executable, str(server_path)]
    server_proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    print('Servidor iniciado (PID):', server_proc.pid)
    def reader():
        try:
            for line in server_proc.stdout:
                print('[SERVER]', line.rstrip())
        except Exception as e:
            print('Reader error:', e)
    t = threading.Thread(target=reader, daemon=True)
    t.start()
    return server_proc

# Inicie o servidor com: start_server()

## 5) Teste de cliente programático via socket (envio/recebimento)
Função helper que conecta ao servidor local e envia mensagens no formato `FROM|TO|AMOUNT`.

In [ ]:
def send_request(msg, host='127.0.0.1', port=6000, timeout=5):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.settimeout(timeout)
    try:
        s.connect((host, port))
        s.send(msg.encode())
        data = s.recv(8192).decode()
        return data
    finally:
        try:
            s.close()
        except:
            pass

# Exemplos (execute após iniciar o servidor):
# print(send_request('USD|BRL|10'))
# print(send_request('XYZ|BRL|10'))  # moeda inválida

## 6) Testes unitários básicos com pytest: convert_currency e fallback
Esta célula escreve um arquivo de testes simples em `tests/test_server.py` e executa pytest de forma programática.

In [ ]:
tests_dir = base_path / 'tests'
tests_dir.mkdir(exist_ok=True)
test_file = tests_dir / 'test_server.py'
test_code = '''
import pytest
from importlib import util
spec = util.spec_from_file_location('servidor_tests', r'{}')
test_code += '''
servidor = util.module_from_spec(spec)
spec.loader.exec_module(servidor)

def test_fallback_non_empty():
    fr = servidor.get_fallback_rates()
    assert 'USD' in fr

def test_convert_invalid_format():
    assert 'ERRO' in servidor.convert_currency('BAD|FORMAT')
'''
: null,
: []
: 
,
: {
: 
,
: 

: [
,
,
,

: null,
: []
: 
,
: {
: 
,
: 

: [
7
,

: 
,
: {
: 
,
: 

: [
,
,
,
,

: null,
: []
: 
,
: {
: 
,
: 

: [
8
,

: 
,
: {
: 
,
: 

: [
,
,
,

: null,
: []
: 
,
: {
: 
,
: 

: [
9
,
,
,

In [ ]:
# === Conteúdo completo de servidor.py ===
# Cole aqui o código completo de servidor.py para referência e edição direta no notebook
from socket import *
import requests
import json
import threading
from datetime import datetime, timedelta

serverPort = 6000

rates_cache = {
    'rates': {},
    'last_update': None,
    'cache_duration': 3600
}

cache_lock = threading.Lock()

client_counter = 0
active_clients = 0
clients_lock = threading.Lock()

# Retorna as taxas de hoje do Banco Central do Brasil de Dólar e Euro para Real
def get_bcb_rates():
    
    try:
        today = datetime.now()
        date_str = today.strftime('%m-%d-%Y')
        
        urls = {
            'USD': f'https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarDia(dataCotacao=@dataCotacao)?@dataCotacao=%27{date_str}%27&$format=json',
            'EUR': f'https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoMoedaDia(moeda=@moeda,dataCotacao=@dataCotacao)?@moeda=%27EUR%27&@dataCotacao=%27{date_str}%27&$format=json',
        }
        
        rates = {'BRL': 1.0}
        
        try:
            resp = requests.get(urls['USD'], timeout=5)
            if resp.status_code == 200:
                data = resp.json()
                if data.get('value'):
                    rates['USD'] = 1.0 / data['value'][0]['cotacaoCompra']
                    print(f'[BCB] USD: {data["value"][0]["cotacaoCompra"]:.4f} BRL')
        except:
            pass
        
        try:
            resp = requests.get(urls['EUR'], timeout=5)
            if resp.status_code == 200:
                data = resp.json()
                if data.get('value'):
                    rates['EUR'] = 1.0 / data['value'][0]['cotacaoCompra']
                    print(f'[BCB] EUR: {data["value"][0]["cotacaoCompra"]:.4f} BRL')
        except:
            pass
        
        return rates
        
    except Exception as e:
        print(f'[BCB] Erro: {e}')
        return {}

# Combina a ExchangeRate-API com o BCB para obter taxas de câmbio atualizadas.
# Primeiro verifica o cache para evitar chamadas excessivas à API, mas se o cache estiver expirado (1 hora), busca novas taxas.
# Depois tenta obter a cotação pela ExchangeRate-API e substitui a taxa do BRL pela do BCB, se disponível.
# Senão, usa o fallback.
def get_exchange_rates():
    
    with cache_lock:
        now = datetime.now()
        
        if (rates_cache['last_update'] and 
            rates_cache['rates'] and
            (now - rates_cache['last_update']).seconds < rates_cache['cache_duration']):
            print('[CACHE] Usando taxas em cache')
            return rates_cache['rates'], rates_cache['last_update']
        
        print('[API] Buscando taxas atualizadas...')
        
        try:
            
            url = 'https://api.exchangerate-api.com/v4/latest/USD'
            response = requests.get(url, timeout=5)
            
            if response.status_code == 200:
                data = response.json()
                rates = data['rates']
                
                bcb_rates = get_bcb_rates()
                if 'BRL' in bcb_rates and bcb_rates['BRL'] != 1.0:
                    rates['BRL'] = 1.0 / bcb_rates['BRL']
                    print('[BCB] ✅ Usando cotação oficial do Banco Central para BRL')
                
                rates_cache['rates'] = rates
                rates_cache['last_update'] = now
                
                print(f'[API] ✅ {len(rates)} moedas disponíveis')
                
                return rates, now
            else:
                return get_fallback_rates(), now
                
        except Exception as e:
            print(f'[API] Erro: {e}')
            return get_fallback_rates(), now

# Taxas de câmbio fallback
def get_fallback_rates():
    return {
        'USD': 1.0,
        'BRL': 5.12,
        'EUR': 0.92,
        'GBP': 0.79,
        'JPY': 149.50,
        'CAD': 1.36,
        'AUD': 1.53,
        'CHF': 0.88,
        'CNY': 7.24,
        'ARS': 350.00,
        'MXN': 17.20,
        'CLP': 890.00
    }

# Processa a mensagem de conversão recebida do cliente
# Espera o formato <moeda_origem>|<moeda_destino>|<valor>
# Retorna a resposta no formato:
# SUCESSO|<moeda_origem>|<moeda_destino>|<valor>|<resultado>|<taxa>|<data_atualizacao>|<fonte>
# Ou
# ERRO:<mensagem_erro>
def convert_currency(message):

    try:
        parts = message.split('|')
        
        if len(parts) != 3:
            return "ERRO: Formato inválido. Use: FROM|TO|AMOUNT"
        
        from_curr = parts[0].strip().upper()
        to_curr = parts[1].strip().upper()
        amount = float(parts[2].strip())
        
        rates, last_update = get_exchange_rates()
        
        if from_curr not in rates:
            return f"ERRO: Moeda {from_curr} não suportada"
        
        if to_curr not in rates:
            return f"ERRO: Moeda {to_curr} não suportada"
        
        if amount <= 0:
            return "ERRO: Valor deve ser maior que zero"
        
        usd_amount = amount / rates[from_curr]
        result = usd_amount * rates[to_curr]
        rate = rates[to_curr] / rates[from_curr]
        
        update_time = last_update.strftime('%Y-%m-%d %H:%M:%S')
        
        source = "BCB+API" if (to_curr == 'BRL' or from_curr == 'BRL') else "API"
        
        response = f"SUCESSO|{from_curr}|{to_curr}|{amount:.2f}|{result:.2f}|{rate:.6f}|{update_time}|{source}"
        return response
        
    except ValueError:
        return "ERRO: Valor inválido"
    except Exception as e:
        return f"ERRO: {str(e)}"

# Função que trata cada cliente em uma thread separada
# Permite múltiplas conexões simultâneas
# Primeiro incrementa o contador de clientes ativos
# Depois processa a mensagem recebida e envia a resposta
# Finalmente decrementa o contador de clientes ativos ao desconectar
def handle_client(connectionSocket, addr, client_id):
    
    global active_clients
    
    with clients_lock:
        active_clients += 1
    
    print(f'\n[CONECTADO] Cliente #{client_id} - {addr[0]}:{addr[1]}')
    print(f'[INFO] Clientes ativos: {active_clients}')
    
    try:
        message = connectionSocket.recv(1024).decode()
        
        if not message:
            print(f'[AVISO] Cliente #{client_id} enviou mensagem vazia')
            return
        
        print(f'[RECEBIDO] Cliente #{client_id}: {message}')
        
        response = convert_currency(message)
        print(f'[ENVIANDO] Cliente #{client_id}: {response.split("|")[0]}')
        
        connectionSocket.send(response.encode())
        
    except Exception as e:
        print(f'[ERRO] Cliente #{client_id}: {e}')
        try:
            error_msg = f"ERRO: {str(e)}"
            connectionSocket.send(error_msg.encode())
        except:
            pass
    
    finally:
        connectionSocket.close()
        
        with clients_lock:
            active_clients -= 1
        
        print(f'[DESCONECTADO] Cliente #{client_id} - {addr[0]}:{addr[1]}')
        print(f'[INFO] Clientes ativos: {active_clients}')

serverSocket = socket(AF_INET, SOCK_STREAM)
serverSocket.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
serverSocket.bind(('', serverPort))
serverSocket.listen(10)  # Até 10 conexões na fila

print('=' * 70)
print('💱 SERVIDOR CONVERSOR - MULTI-THREAD (BCB + API)')
print('=' * 70)
print(f'📡 Porta: {serverPort}')
print(f'🔧 Modo: Multi-thread (suporta múltiplos clientes)')
print('⏳ Carregando taxas iniciais...')

initial_rates, _ = get_exchange_rates()

print(f'✅ {len(initial_rates)} moedas carregadas')
print('🌍 Principais: USD, BRL, EUR, GBP, JPY, CAD, AUD, CHF, CNY, ARS')
print(f'💾 Cache: {rates_cache["cache_duration"]} segundos')
print('👂 Aguardando conexões...')
print('=' * 70)

try:
    while True:
        
        connectionSocket, addr = serverSocket.accept()
        
        client_counter += 1
        current_client_id = client_counter
        
        client_thread = threading.Thread(
            target=handle_client,
            args=(connectionSocket, addr, current_client_id),
            daemon=True
        )
        
        client_thread.start()
        
        print(f'[THREAD] Thread iniciada para Cliente #{current_client_id}')

except KeyboardInterrupt:
    print('\n\n[SERVIDOR] Encerrando servidor...')
    print(f'[INFO] Total de clientes atendidos: {client_counter}')
    serverSocket.close()
    print('[SERVIDOR] Servidor encerrado com sucesso')
except Exception as e:
    print(f'\n[ERRO FATAL] {e}')
    serverSocket.close()


In [ ]:
# === Conteúdo completo de cliente.py ===
# Cole aqui o código completo de cliente.py para referência e edição direta no notebook
import argparse
import os
from socket import socket, AF_INET, SOCK_STREAM
import sys


def valid_currency(code: str) -> str:
    code = code.strip().upper()
    if len(code) != 3 or not code.isalpha():
        raise ValueError('Código de moeda deve ter 3 letras (ex: USD, BRL)')
    return code


def prompt_currency(prompt_text: str) -> str:
    while True:
        try:
            val = input(prompt_text).strip()
            return valid_currency(val)
        except ValueError as e:
            print(f'Entrada inválida: {e}')


def prompt_amount(prompt_text: str) -> float:
    while True:
        val = input(prompt_text).strip().replace(',', '.')
        try:
            amount = float(val)
            if amount <= 0:
                print('O valor deve ser maior que zero')
                continue
            return amount
        except ValueError:
            print('Valor inválido. Use um número, ex: 100 ou 12.50')


def build_message(from_curr: str, to_curr: str, amount: float) -> str:
    # mantém o formato esperado pelo servidor: FROM|TO|AMOUNT
    return f"{from_curr}|{to_curr}|{amount:.2f}"


def main():
    # Carrega variáveis do arquivo .env (se existir) para encapsular o IP/PORT
    def load_dotenv(path='.env'):
        try:
            if not os.path.exists(path):
                return
            with open(path, 'r', encoding='utf-8') as f:
                for line in f:
                    line = line.strip()
                    if not line or line.startswith('#'):
                        continue
                    if '=' in line:
                        k, v = line.split('=', 1)
                        k = k.strip()
                        v = v.strip().strip('"').strip("'")
                        if k and v and k not in os.environ:
                            os.environ[k] = v
        except Exception:
            pass

    load_dotenv()

    parser = argparse.ArgumentParser(description='Cliente conversor de moedas')
    parser.add_argument('server', nargs='?', default=os.environ.get('SERVER', '127.0.0.1'),
                        help='IP do servidor (padrão: 127.0.0.1 ou definido em .env)')
    parser.add_argument('--port', '-p', type=int, default=int(os.environ.get('PORT', '6000')),
                        help='Porta do servidor (padrão: 6000 ou definido em .env)')

    args = parser.parse_args()

    serverName = args.server
    serverPort = args.port

    print('=' * 60)
    print('💱 CLIENTE CONVERSOR DE MOEDAS')
    print('=' * 60)
    print(f'Conectando a {serverName}:{serverPort}...')

    clientSocket = socket(AF_INET, SOCK_STREAM)

    try:
        clientSocket.connect((serverName, serverPort))
        print('✅ Conectado ao servidor!\n')

        print('MOEDAS DISPONÍVEIS (exemplos):')
        print('  USD - Dólar Americano')
        print('  BRL - Real Brasileiro')
        print('  EUR - Euro')
        print('  GBP - Libra Esterlina')
        print('  JPY - Iene Japonês')
        print()

        # perguntas separadas
        print('Preencha os dados da conversão:')
        from_curr = prompt_currency('  Moeda origem (ex: USD): ')
        to_curr = prompt_currency('  Moeda destino (ex: BRL): ')
        amount = prompt_amount('  Valor (ex: 100.00): ')

        message = build_message(from_curr, to_curr, amount)

        print('\n⏳ Enviando requisição...')
        clientSocket.send(message.encode())

        response = clientSocket.recv(4096).decode()

        print('\n' + '=' * 60)
        print('📊 RESULTADO DA CONVERSÃO')
        print('=' * 60)

        parts = response.split('|')

        if parts[0] == 'SUCESSO':
            from_curr = parts[1]
            to_curr = parts[2]
            amount = float(parts[3])
            result = float(parts[4])
            rate = float(parts[5])

            print(f'\n  De:        {from_curr}')
            print(f'  Valor:     {amount:.2f}')
            print(f'\n  Para:      {to_curr}')
            print(f'  Resultado: {result:.2f}')
            print(f'\n  Taxa:      1 {from_curr} = {rate:.6f} {to_curr}')
        else:
            print(f'\n❌ {response}')

        print('\n' + '=' * 60)

    except ConnectionRefusedError:
        print('❌ Erro: Não foi possível conectar ao servidor')
        print('   Verifique se o servidor está rodando e o IP/porta estão corretos')
        sys.exit(1)
    except Exception as e:
        print(f'❌ Erro: {e}')
        sys.exit(1)
    finally:
        try:
            clientSocket.close()
        except:
            pass
        print('\n👋 Desconectado')


if __name__ == '__main__':
    main()


In [ ]:
# --- Versão organizada de `servidor.py` (apenas para referência/edit no notebook) ---
"""
Versão modular e segura do servidor para referência no notebook.
Não inicia o servidor automaticamente ao ser importada.
Use a função `run_server()` ou execute a célula apropriada para iniciar em background.
"""

import threading
from socket import AF_INET, SOCK_STREAM, socket, SOL_SOCKET, SO_REUSEADDR
from datetime import datetime
import requests

SERVER_PORT = 6000

rates_cache = {
    'rates': {},
    'last_update': None,
    'cache_duration': 3600
}
cache_lock = threading.Lock()


def get_fallback_rates():
    """Retorna um dicionário com taxas fallback."""
    return {
        'USD': 1.0,
        'BRL': 5.12,
        'EUR': 0.92,
        'GBP': 0.79,
        'JPY': 149.50,
        'CAD': 1.36,
        'AUD': 1.53,
        'CHF': 0.88,
        'CNY': 7.24,
        'ARS': 350.00,
        'MXN': 17.20,
        'CLP': 890.00
    }


def get_bcb_rates():
    """Tenta obter taxas do BCB para USD e EUR. Retorna dicionário com BRL=1.0 por padrão."""
    try:
        today = datetime.now()
        date_str = today.strftime('%m-%d-%Y')
        urls = {
            'USD': f"https://olinda.bcb.gov.br/.../CotacaoDolarDia(dataCotacao=@dataCotacao)?@dataCotacao=%27{date_str}%27&$format=json",
            'EUR': f"https://olinda.bcb.gov.br/.../CotacaoMoedaDia(moeda=@moeda,dataCotacao=@dataCotacao)?@moeda=%27EUR%27&@dataCotacao=%27{date_str}%27&$format=json",
        }
        rates = {'BRL': 1.0}
        # Chamadas com timeout e proteção para evitar falhas fatais
        try:
            resp = requests.get(urls['USD'], timeout=5)
            if resp.status_code == 200:
                data = resp.json()
                if data.get('value'):
                    rates['USD'] = 1.0 / data['value'][0]['cotacaoCompra']
        except Exception:
            pass
        try:
            resp = requests.get(urls['EUR'], timeout=5)
            if resp.status_code == 200:
                data = resp.json()
                if data.get('value'):
                    rates['EUR'] = 1.0 / data['value'][0]['cotacaoCompra']
        except Exception:
            pass
        return rates
    except Exception:
        return {}


def get_exchange_rates():
    """Retorna (rates, last_update). Usa cache e faz fallback em caso de erro."""
    with cache_lock:
        now = datetime.now()
        last = rates_cache.get('last_update')
        if last and rates_cache.get('rates') and (now - last).seconds < rates_cache['cache_duration']:
            return rates_cache['rates'], last
        try:
            url = 'https://api.exchangerate-api.com/v4/latest/USD'
            resp = requests.get(url, timeout=5)
            if resp.status_code == 200:
                data = resp.json()
                rates = data.get('rates', {})
                # tentar mesclar BCB para BRL
                bcb = get_bcb_rates()
                if 'BRL' in bcb and bcb['BRL'] != 1.0:
                    rates['BRL'] = 1.0 / bcb['BRL']
                rates_cache['rates'] = rates
                rates_cache['last_update'] = now
                return rates, now
            else:
                return get_fallback_rates(), now
        except Exception:
            return get_fallback_rates(), now


def convert_currency(message: str) -> str:
    """Processa mensagem 'FROM|TO|AMOUNT' e retorna resposta padronizada."""
    try:
        parts = message.split('|')
        if len(parts) != 3:
            return 'ERRO: Formato inválido. Use: FROM|TO|AMOUNT'
        src, dst, val = parts[0].strip().upper(), parts[1].strip().upper(), parts[2].strip()
        amount = float(val)
        rates, last = get_exchange_rates()
        if src not in rates:
            return f'ERRO: Moeda {src} não suportada'
        if dst not in rates:
            return f'ERRO: Moeda {dst} não suportada'
        if amount <= 0:
            return 'ERRO: Valor deve ser maior que zero'
        usd = amount / rates[src]
        result = usd * rates[dst]
        rate = rates[dst] / rates[src]
        update_time = last.strftime('%Y-%m-%d %H:%M:%S') if last else ''
        source = 'BCB+API' if ('BRL' in (src, dst)) else 'API'
        return f'SUCESSO|{src}|{dst}|{amount:.2f}|{result:.2f}|{rate:.6f}|{update_time}|{source}'
    except ValueError:
        return 'ERRO: Valor inválido'
    except Exception as e:
        return f'ERRO: {e}'


def handle_client(conn, addr, client_id):
    """Handler simples: lê uma mensagem, processa e responde."""
    try:
        msg = conn.recv(1024).decode()
        if not msg:
            return
        resp = convert_currency(msg)
        conn.send(resp.encode())
    finally:
        try:
            conn.close()
        except Exception:
            pass


def run_server(host: str = '', port: int = SERVER_PORT, backlog: int = 10):
    """Cria e executa o loop do servidor (bloqueante)."""
    serv = socket(AF_INET, SOCK_STREAM)
    serv.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
    serv.bind((host, port))
    serv.listen(backlog)
    print(f'Servidor escutando em {host or "0.0.0.0"}:{port}')
    client_id = 0
    try:
        while True:
            conn, addr = serv.accept()
            client_id += 1
            t = threading.Thread(target=handle_client, args=(conn, addr, client_id), daemon=True)
            t.start()
    except KeyboardInterrupt:
        serv.close()


# Nota: não executamos run_server() automaticamente ao importar este código no notebook

# Opcional: grava a versão organizada no disco (sobrescreve se autorizado)
def write_server_file(path):
    """Escreve uma versão organizada do servidor em `path`. Use com cuidado."""
    text = '''"""Arquivo organizado gerado no notebook"""\n\n'''
    # Para economizar espaço não escrevemos todo o texto automaticamente aqui; podemos montar e gravar se desejar
    with open(path, 'w', encoding='utf-8') as f:
        f.write('# arquivo gerado a partir do notebook - substitua com cautela\n')
    print('Arquivo escrito em', path)


In [ ]:
# --- Versão organizada de `cliente.py` (apenas para referência/edit no notebook) ---
"""
Versão limpa do cliente para uso no notebook. A função `run_client()` implementa o fluxo principal
mas não é executada automaticamente ao importar o código.
"""

import argparse
import os
from socket import socket, AF_INET, SOCK_STREAM
import sys


def valid_currency(code: str) -> str:
    code = code.strip().upper()
    if len(code) != 3 or not code.isalpha():
        raise ValueError('Código de moeda deve ter 3 letras (ex: USD, BRL)')
    return code


def build_message(from_curr: str, to_curr: str, amount: float) -> str:
    return f"{from_curr}|{to_curr}|{amount:.2f}"


def run_client_interactive(server='127.0.0.1', port=6000):
    """Fluxo interativo: lê inputs e realiza conversão."""
    s = socket(AF_INET, SOCK_STREAM)
    try:
        s.connect((server, port))
        print('Conectado a', server, port)
        from_curr = input('Moeda origem (ex: USD): ').strip().upper()
        to_curr = input('Moeda destino (ex: BRL): ').strip().upper()
        amount = float(input('Valor: ').strip().replace(',', '.'))
        msg = build_message(from_curr, to_curr, amount)
        s.send(msg.encode())
        resp = s.recv(4096).decode()
        print('Resposta do servidor:')
        print(resp)
    finally:
        try:
            s.close()
        except:
            pass


def client_send_message(server, port, msg, timeout=5):
    """Helper programático para testes: envia e recebe uma resposta."""
    s = socket(AF_INET, SOCK_STREAM)
    s.settimeout(timeout)
    try:
        s.connect((server, port))
        s.send(msg.encode())
        return s.recv(8192).decode()
    finally:
        try:
            s.close()
        except:
            pass


if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('server', nargs='?', default=os.environ.get('SERVER','127.0.0.1'))
    parser.add_argument('--port','-p', type=int, default=int(os.environ.get('PORT', '6000')))
    args = parser.parse_args()
    run_client_interactive(args.server, args.port)
